# **How to Get insightful information about a Stock performance in one click with code interpreter agent using LlamaIndex and Anthropic**




*  The goal of this agent is to request from an LLM to propose a python code to fetch historical prices for a given stock on the current month, and get analysis based on these prices.

*  I built this agent using FunctionCallingAgent abstraction from LlamaIndex, and Code Interpreter tool.

*  Trick: I asked first for a code to detect which date is today, and then instruct it to fetch historical prices based on this date. If you don't do that, It will take


In [ ]:
%pip install llama-index-llms-anthropic
%pip install llama-index-tools-code-interpreter
%pip install llama-index -q

In [2]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

import os
os.environ["ANTHROPIC_API_KEY"] = CLAUDE_API_KEY

In [3]:
from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec

In [25]:
code_spec = CodeInterpreterToolSpec()

tools = code_spec.to_tool_list()

# Specify the Agent:

Using FunctionCallingAgent to build our agent. This object allows wokring with an LLM that supports function calling like Anthropic.

In the tools, we'll specify the code interpreter tool, which is a LLamaIndex package.

In [59]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

llm_claude = Anthropic(model="claude-3-5-sonnet-20241022")

from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(
    tools,
    llm=llm_claude,
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [38]:
stock = 'TESLA'

prompt = f"""
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of {stock} from the beginning of the month until today.
- Analyze the last month prices
"""

resp = agent.chat(prompt)

> Running step 8ec2fa1a-d8ef-439e-904b-df90215aea68. Step input: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of Tesla from the beginning of the month until today.
- Analyze the last month prices

Added user message to memory: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of Tesla from the beginning of the month until today.
- Analyze the last month prices

=== LLM Response ===
I'll help you write a Python code to analyze Tesla's stock prices. We'll use `yfinance` for fetching stock data and `pandas` for analysis. Here's the code:
=== Calling Function ===
Calling function: code_interpreter with args: {"code": "import yfinance as yf\nimport pandas as pd\nfrom datetime import datetime, date\n\n# Get today's date\ntoday = date.today()\nprint(f\"Today's date is: {today}\")\n\n# Get the first day of the current month\nstart_date = today.replace(day=1)\nprint(f\"Start date is: {star

In [41]:
print(resp.response)

The code is now working correctly! Let me explain what the analysis shows:

1. Date Information:
   - Today's date: 2024-11-17
   - Analysis period: From 2024-11-01 to 2024-11-17

2. Price Analysis:
   - Starting Price: $248.98
   - Ending Price: $320.72
   - Highest Price: $358.64
   - Lowest Price: $238.88

3. Performance Metrics:
   - Average Daily Return: 2.76%
   - Daily Return Volatility: 6.64%
   - Total Return for the period: 28.81%

4. Statistical Summary:
   - Mean Price: $299.14
   - Standard Deviation: $36.87
   - Median (50%): $311.18
   - 25th percentile: $269.99
   - 75th percentile: $324.85

The analysis shows that Tesla stock has had a strong positive performance during this period, with a total return of 28.81%. The stock has shown significant volatility (6.64% daily standard deviation) but has maintained an upward trend with an average daily return of 2.76%.

Note: There are some deprecation warnings in the code related to floating point conversion, but they don't af

Access to the Agent Memory

In [43]:
agent.memory

ChatMemoryBuffer(chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='\nWrite a python code to :\n- Detect which date is today\n- Based on this date, fetch historical prices of Tesla from the beginning of the month until today.\n- Analyze the last month prices\n', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="I'll help you write a Python code to analyze Tesla's stock prices. We'll use `yfinance` for fetching stock data and `pandas` for analysis. Here's the code:", additional_kwargs={'tool_calls': [{'id': 'toolu_01X9E5U1aeXk8BnjrdVYXVWK', 'input': {'code': 'import yfinance as yf\nimport pandas as pd\nfrom datetime import datetime, date\n\n# Get today\'s date\ntoday = date.today()\nprint(f"Today\'s date is: {today}")\n\n# Get the first day of the current month\nstart_date = today.replace(day=1)\nprint(f"Start date is: {start_date}")\n\n# Fetch Tesla stock data\ntesla = yf.download(\'TSLA\', start

In [58]:
import yfinance as yf
import pandas as pd
from datetime import datetime, date

# Get today's date
today = date.today()
print(f"Today's date is: {today}")

# Get the first day of the current month
start_date = today.replace(day=1)
print(f"Start date is: {start_date}")

# Fetch Tesla stock data
tesla = yf.download('TSLA', start=start_date, end=today)
print("\nTesla stock data shape:", tesla.shape)

# Basic analysis of the stock prices
print("\nPrice Summary Statistics:")
print(tesla['Close'].describe())

# Calculate daily returns
tesla['Daily_Return'] = tesla['Close'].pct_change()

# Print key metrics
print("\nKey Metrics:")
print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
print(f"Lowest Price: ${float(tesla['Low'].min()):.2f}")
print(f"Average Daily Return: {float(tesla['Daily_Return'].mean()*100):.2f}%")
print(f"Daily Return Volatility: {float(tesla['Daily_Return'].std()*100):.2f}%")

# Calculate total return for the period
total_return = float(((tesla['Close'].iloc[-1] - tesla['Close'].iloc[0]) / tesla['Close'].iloc[0]) * 100)
print(f"\nTotal Return for the period: {total_return:.2f}%")

Today's date is: 2024-11-17
Start date is: 2024-11-01


[*********************100%***********************]  1 of 1 completed


Tesla stock data shape: (11, 6)

Price Summary Statistics:
Ticker        TSLA
count    11.000000
mean    299.140907
std      36.867474
min     242.839996
25%     269.985001
50%     311.179993
75%     324.854996
max     350.000000

Key Metrics:
Starting Price: $248.98
Ending Price: $320.72
Highest Price: $358.64
Lowest Price: $238.88
Average Daily Return: 2.76%
Daily Return Volatility: 6.64%

Total Return for the period: 28.81%



<ipython-input-58-668e99dcbffa>:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
<ipython-input-58-668e99dcbffa>:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
<ipython-input-58-668e99dcbffa>:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
<ipython-input-58-668e99dcbffa>:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Lowest Price: ${float(tesla['Low'].min()):.2f}")
<ipython-input-58-668e99dcbffa>:34